In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk

In [2]:
df=pd.read_csv('/content/drive/MyDrive/Musical_instruments_reviews.csv')

In [3]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10261 entries, 0 to 10260
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      10261 non-null  object 
 1   asin            10261 non-null  object 
 2   reviewerName    10234 non-null  object 
 3   helpful         10261 non-null  object 
 4   reviewText      10254 non-null  object 
 5   overall         10261 non-null  float64
 6   summary         10261 non-null  object 
 7   unixReviewTime  10261 non-null  int64  
 8   reviewTime      10261 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 721.6+ KB


In [5]:
df.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [6]:
#Checking the Null Values
df.isnull().sum()

reviewerID         0
asin               0
reviewerName      27
helpful            0
reviewText         7
overall            0
summary            0
unixReviewTime     0
reviewTime         0
dtype: int64

In [7]:
df['reviewText'].fillna("", inplace = True)

In [8]:
df['reviewText'].fillna("", inplace = True)

In [9]:
df['review']=df['reviewText']+' '+df['summary']

In [10]:
df['review'][0]

"Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing, good"

In [11]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,review
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014","Not much to write about here, but it does exac..."
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013",The product does exactly as it should and is q...
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013",The primary job of this device is to block the...
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014",Nice windscreen protects my MXL mic and preven...
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014",This pop filter is great. It looks and perform...


In [12]:
df = df.drop(['reviewerID','asin','helpful','reviewText','reviewerName','unixReviewTime','reviewTime','summary'],axis=1)

In [13]:
df.columns

Index(['overall', 'review'], dtype='object')

In [14]:
df['overall'].value_counts()

5.0    6938
4.0    2084
3.0     772
2.0     250
1.0     217
Name: overall, dtype: int64

In [15]:
def rating(rating):
    if(int(rating) == 1 or int(rating) == 2 or int(rating) == 3):
        return 0
    else: 
        return 1
df.overall = df.overall.apply(rating)

In [16]:
df['overall'].value_counts()

1    9022
0    1239
Name: overall, dtype: int64

In [17]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
import re

In [18]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
import string
from string import punctuation

In [20]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [21]:
from nltk.corpus import wordnet

In [22]:
def get_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [23]:
lemmatizer = WordNetLemmatizer()

In [24]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [25]:
from nltk import pos_tag

In [26]:
def lemmatize_words(review):
    final_review = []
    for i in review.split():
        if i.strip().lower() not in stop:
            pos = pos_tag([i.strip()])
            word = lemmatizer.lemmatize(i.strip(),get_pos(pos[0][1]))
            final_review.append(word.lower())
    return " ".join(final_review)

In [27]:
df['review'] = df.review.apply(lemmatize_words)

In [28]:
df['review']

0        much write here, exactly suppose to. filter po...
1        product exactly quite affordable.i realize dou...
2        primary job device block breath would otherwis...
3        nice windscreen protects mxl mic prevents pops...
4        pop filter great. look performs like studio fi...
                               ...                        
10256               great, expected. thank all. five stars
10257    i've think try nanoweb string while, bit put h...
10258    try coat string past include elixirs) never fo...
10259    well, made elixir developed taylor guitars ......
10260    string really quite good, call perfect. unwoun...
Name: review, Length: 10261, dtype: object

In [29]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split

In [30]:
x_train,x_test,y_train,y_test = train_test_split(df.review,df.overall,test_size = 0.2 , random_state = 0)

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
cv = CountVectorizer(min_df= 0,max_df= 1,binary=False,ngram_range= (1,3))

cv_train_reviews = cv.fit_transform(x_train)

cv_test_reviews = cv.transform(x_test)

print('cv_train:',cv_train_reviews.shape)
print('cv_test:',cv_test_reviews.shape)

cv_train: (8208, 573616)
cv_test: (2053, 573616)


In [33]:
lr = LogisticRegression(penalty= 'l2',max_iter=500,C = 1,random_state= 0)
 
lr_bow = lr.fit(cv_train_reviews,y_train)
print(lr_bow)

lr_tfidf = lr.fit(cv_train_reviews,y_train)
print(lr_tfidf)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [34]:
lr_bow_predict = lr.predict(cv_test_reviews)

lr_tfidf_predict = lr.predict(cv_test_reviews)

In [35]:
lr_bow_score = accuracy_score(y_test,lr_bow_predict)
print("lr_bow_score :",lr_bow_score)
#Accuracy score
lr_tfidf_score = accuracy_score(y_test,lr_tfidf_predict)
print("lr_tfidf_score :",lr_tfidf_score)

lr_bow_score : 0.8899171943497322
lr_tfidf_score : 0.8899171943497322


In [36]:
lr_bow_report = classification_report(y_test,lr_bow_predict,target_names=['0','1'])
print(lr_bow_report)

#Classification report 
lr_tfidf_report = classification_report(y_test,lr_tfidf_predict,target_names=['0','1'])
print(lr_tfidf_report)

              precision    recall  f1-score   support

           0       1.00      0.01      0.02       228
           1       0.89      1.00      0.94      1825

    accuracy                           0.89      2053
   macro avg       0.94      0.50      0.48      2053
weighted avg       0.90      0.89      0.84      2053

              precision    recall  f1-score   support

           0       1.00      0.01      0.02       228
           1       0.89      1.00      0.94      1825

    accuracy                           0.89      2053
   macro avg       0.94      0.50      0.48      2053
weighted avg       0.90      0.89      0.84      2053



In [37]:
mnb = MultinomialNB()

mnb_bow = mnb.fit(cv_train_reviews,y_train)
print(mnb_bow)

mnb_tfidf = mnb.fit(cv_train_reviews,y_train)
print(mnb_tfidf)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


In [38]:
mnb_bow_predict = mnb.predict(cv_test_reviews)

mnb_tfidf_predict = mnb.predict(cv_test_reviews)

In [41]:
mnb_bow_score = accuracy_score(y_test,mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)
mnb_tfidf_score = accuracy_score(y_test,mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)

mnb_bow_score : 0.8402338041889917
mnb_tfidf_score : 0.8402338041889917


In [42]:
mnb_bow_report = classification_report(y_test,mnb_bow_predict,target_names = ['0','1'])
print(mnb_bow_report)
mnb_tfidf_report = classification_report(y_test,mnb_tfidf_predict,target_names = ['0','1'])
print(mnb_tfidf_report)

              precision    recall  f1-score   support

           0       0.21      0.16      0.18       228
           1       0.90      0.92      0.91      1825

    accuracy                           0.84      2053
   macro avg       0.56      0.54      0.55      2053
weighted avg       0.82      0.84      0.83      2053

              precision    recall  f1-score   support

           0       0.21      0.16      0.18       228
           1       0.90      0.92      0.91      1825

    accuracy                           0.84      2053
   macro avg       0.56      0.54      0.55      2053
weighted avg       0.82      0.84      0.83      2053



In [43]:
ls =  SVC()
ls_bow = ls.fit(cv_train_reviews,y_train)
ls_tfidf = ls.fit(cv_train_reviews,y_train)

In [44]:
#predict

ls_bow_predict = ls.predict(cv_test_reviews)
ls_tfidf_predict = ls.predict(cv_test_reviews)


In [45]:
#Accuracy

ls_bow_score = accuracy_score(y_test, ls_bow_predict)
ls_tfidf_score = accuracy_score(y_test,ls_tfidf_predict)

print('ls bow accuracy:', ls_bow_score)
print('ls tfidf accuracy:', ls_tfidf_score)

ls bow accuracy: 0.8889430102289333
ls tfidf accuracy: 0.8889430102289333


In [46]:
#Classification Report
ls_bow_report = classification_report(y_test,ls_bow_predict,target_names = ['0','1'])
print(ls_bow_report)
ls_tfidf_report = classification_report(y_test,ls_tfidf_predict,target_names = ['0','1'])
print(ls_tfidf_report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       228
           1       0.89      1.00      0.94      1825

    accuracy                           0.89      2053
   macro avg       0.44      0.50      0.47      2053
weighted avg       0.79      0.89      0.84      2053

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       228
           1       0.89      1.00      0.94      1825

    accuracy                           0.89      2053
   macro avg       0.44      0.50      0.47      2053
weighted avg       0.79      0.89      0.84      2053



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
